In [1]:
import talib
import numpy as np
import pandas as pd
import random

In [2]:
file_path = '/Users/mymac/Google_Drive/Forex_Robot/'

In [3]:
df = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2020-2021.csv')
df.Date = pd.to_datetime(df.Date)

df.reset_index(drop=True, inplace=True)

In [4]:
print(df.head())
print('-' * 75)
print(df.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2020-07-30 07:00:00   1.29543   1.29607  1.29540    1.29572   1.29562   
1 2020-07-30 07:05:00   1.29569   1.29591  1.29505    1.29542   1.29588   
2 2020-07-30 07:10:00   1.29541   1.29590  1.29514    1.29559   1.29557   
3 2020-07-30 07:15:00   1.29561   1.29567  1.29510    1.29515   1.29577   
4 2020-07-30 07:20:00   1.29514   1.29643  1.29501    1.29642   1.29529   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  
0   1.29623  1.29560    1.29589   1.29552   1.29615  1.29550    1.29580  
1   1.29608  1.29522    1.29560   1.29578   1.29600  1.29514    1.29551  
2   1.29606  1.29535    1.29575   1.29549   1.29598  1.29524    1.29567  
3   1.29584  1.29530    1.29533   1.29569   1.29576  1.29520    1.29524  
4   1.29660  1.29518    1.29660   1.29522   1.29652  1.29510    1.29651  
---------------------------------------------------------------------------
                     Date  Bi

In [5]:
df['macd'], df['macdsignal'], df['macdhist'] = talib.MACD(df['Mid_Close'])
df['slowk'], df['slowd'] = talib.STOCH(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], fastk_period=14, slowk_period=1, slowd_period=3)
df['ema200'] = talib.EMA(df['Mid_Close'], timeperiod=200)
df['b_upper'], df['b_mid'], df['b_lower'] = talib.BBANDS(df['Mid_Close'], timeperiod=20)
df['sar'] = talib.SAR(df['Mid_High'], df['Mid_Low'])
df['adx'] = talib.ADX(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], timeperiod=14)
df['rsi'] = talib.RSI(df['Mid_Close'], timeperiod=14)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [6]:
value_per_pip = 1.0
# max_pips_to_risk = 0.0020
amounts_per_day = [-0.00008, -0.0001, -0.00012]

In [7]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee


def run_simulation(risk_reward_ratio, spread_cutoff, use_tl, min_pips_to_risk, max_pips_to_risk, adx_cutoff):
    max_pips_to_risk /= 10000
    min_pips_to_risk /= 10000
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    n_units = 50000
    trade = None

    i = 6

    while i < len(df):
        curr_date = df.loc[df.index[i], 'Date']
        ema200_2 = df.loc[df.index[i - 2], 'ema200']
        ema200_1 = df.loc[df.index[i - 1], 'ema200']
        b_mid2 = df.loc[df.index[i - 2], 'b_mid']
        b_mid1 = df.loc[df.index[i - 1], 'b_mid']
        sar2 = df.loc[df.index[i - 2], 'sar']
        sar1 = df.loc[df.index[i - 1], 'sar']
        mid_open2, mid_close2 = df.loc[df.index[i - 2], ['Mid_Open', 'Mid_Close']]
        mid_open1, mid_close1 = df.loc[df.index[i - 1], ['Mid_Open', 'Mid_Close']]
        adx = max(df.loc[df.index[i - 6:i], 'adx'])
        stoch_max = max(df.loc[df.index[i - 6:i], 'slowk'])
        stoch_min = min(df.loc[df.index[i - 6:i], 'slowk'])
        rsi = df.loc[df.index[i - 1], 'rsi']
        curr_ao = df.loc[df.index[i], 'Ask_Open']
        curr_bo = df.loc[df.index[i], 'Bid_Open']
        spread = abs(curr_ao - curr_bo)
        enough_volatility = spread <= spread_cutoff
        macd2, macdsignal2 = df.loc[df.index[i - 2], ['macd', 'macdsignal']]
        macd1, macdsignal1 = df.loc[df.index[i - 1], ['macd', 'macdsignal']]
        trending = adx >= adx_cutoff
        curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
          df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                              'Ask_Low', 'Ask_Close']]

        if trade is None:
          if macd2 < macdsignal2 and macd1 > macdsignal1 and sar2 < min([mid_open2, mid_close2]) and sar1 < min([mid_open1, mid_close1]) and b_mid2 > ema200_2 and b_mid1 > ema200_1 and stoch_min < 20 and enough_volatility and trending:
              all_buys = True

              if all_buys:
                open_price = float(curr_ask_open)
                pullback = None
                pullback = min(df.loc[df.index[i - 6:i], 'Mid_Low'])

                if open_price - pullback < min_pips_to_risk:
                  pullback = open_price - min_pips_to_risk

                elif open_price - pullback > max_pips_to_risk:
                  pullback = open_price - max_pips_to_risk

                if pullback is not None:
                    stop_loss = round(pullback, 5)
                    # stop_loss = round(pullback - pullback_cushion, 3)

                    if stop_loss < open_price:
                        curr_pips_to_risk = open_price - stop_loss

                        if curr_pips_to_risk <= max_pips_to_risk:
                            stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 5)
                            # stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 3)

                            trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                              'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                              'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                            n_buys += 1

                            pips_risked.append(curr_pips_to_risk)

          elif macd2 > macdsignal2 and macd1 < macdsignal1 and sar2 > max([mid_open2, mid_close2]) and sar1 > max([mid_open1, mid_close1]) and b_mid2 < ema200_2 and b_mid1 < ema200_1 and stoch_max > 80 and enough_volatility and trending:
              all_sells = True

              if all_sells:
                open_price = float(curr_bid_open)
                pullback = None
                pullback = max(df.loc[df.index[i - 6:i], 'Mid_High'])

                if pullback - open_price < min_pips_to_risk:
                  pullback = open_price + min_pips_to_risk

                elif pullback - open_price > max_pips_to_risk:
                  pullback = open_price + max_pips_to_risk

                if pullback is not None:
                    stop_loss = round(pullback, 5)
                    # stop_loss = round(pullback + pullback_cushion, 3)

                    if stop_loss > open_price:
                        curr_pips_to_risk = stop_loss - open_price

                        if curr_pips_to_risk <= max_pips_to_risk:
                            stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 5)
                            # stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 3)

                            trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                    'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                    'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                            n_sells += 1

                            pips_risked.append(curr_pips_to_risk)

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
          trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        if use_tl:
          # if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high - trade['pips_risked'] > trade['stop_loss']:
          #     trade['stop_loss'] = curr_bid_high - trade['pips_risked']
          if trade is not None and trade['original_units'] == trade['n_units'] and trade['trade_type'] == 'buy' and curr_bid_high - trade['open_price'] >= trade['pips_risked']:
              trade['stop_loss'] = trade['open_price']
              reward += trade['pips_risked'] * (trade['n_units'] / 2) * value_per_pip
              day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'] / 2)
              trade['n_units'] = trade['n_units'] / 2
              

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
          trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
          trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        if use_tl:
          # if trade is not None and trade['trade_type'] == 'sell' and trade['pips_risked'] + curr_ask_low < trade['stop_loss']:
          #   trade['stop_loss'] = trade['pips_risked'] + curr_ask_low
          if trade is not None and trade['original_units'] == trade['n_units'] and trade['trade_type'] == 'sell' and trade['open_price'] - curr_ask_low >= trade['pips_risked']:
              trade['stop_loss'] = trade['open_price']
              reward += trade['pips_risked'] * (trade['n_units'] / 2) * value_per_pip
              day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'] / 2)
              trade['n_units'] = trade['n_units'] / 2

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
          trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        i += 1

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked

In [9]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
risk_reward_ratio_vals = [1, 1.5, 2]
# risk_reward_ratio_vals = [2]
spread_cutoffs = [2, 2.5]
# spread_cutoffs = [2]
use_tl_vals = [True, False]
min_pips_to_risk_vals = [5, 10]
max_pips_to_risk_vals = [10, 15, 20]
adx_cutoff_vals = [0, 5, 10, 15, 20]
n_possibilities = len(risk_reward_ratio_vals) * len(spread_cutoffs) * len(use_tl_vals) * len(min_pips_to_risk_vals) * len(max_pips_to_risk_vals) * len(adx_cutoff_vals)
all_combos = []

for risk_reward_ratio in risk_reward_ratio_vals:
  for spread_val in spread_cutoffs:
    for use_tl in use_tl_vals:
      for min_pips in min_pips_to_risk_vals:
        for max_pips in max_pips_to_risk_vals:
          for adx_cutoff in adx_cutoff_vals:
            all_combos.append((risk_reward_ratio, spread_val, use_tl, min_pips, max_pips, adx_cutoff))

# percentage_to_try = 0.5
percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_spread_cutoff = None
best_use_tl_val = None
best_min_pips_val = None
best_max_pips_val = None
best_adx_val = None
top_n_results = 20
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, spread_val, use_tl, min_pips, max_pips, adx_cutoff in combos_to_try:
  reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked = run_simulation(risk_reward_ratio, spread_val, use_tl, min_pips, max_pips, adx_cutoff)
  runs_finished += 1

  print(reward)
  print('Num buys: ' + str(n_sells))
  print('Num sells: ' + str(n_buys))
  print('Num trades: ' + str(n_buys + n_sells))
  print('Num wins: ' + str(n_wins))
  print('Num losses: ' + str(n_losses))
  print('Win streak: ' + str(win_streak))
  print('Loss streak: ' + str(loss_streak))
  print('Avg pips risked: ' + str(sum(pips_risked) / len(pips_risked)))
  print('Remaining runs: ' + str(n_runs - runs_finished))

  min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None
  if min_item is None or reward > min_item['reward']:
    if min_item is not None:
      best_rewards.remove(min_item)
    best_rewards.append({'reward': int(reward), 'ratio': risk_reward_ratio, 'spread': spread_val, 'use_tl': use_tl, 'min_pips': min_pips, 'max_pips': max_pips, 'adx_cutoff': adx_cutoff})


  if reward > best_reward:
    best_reward = reward
    best_risk_reward = risk_reward_ratio
    best_spread_cutoff = spread_val
    best_use_tl_val = use_tl
    best_min_pips_val = min_pips
    best_max_pips_val = max_pips
    best_adx_val = adx_cutoff

  print('Best reward so far: ' + str(best_reward))
  print()

Num runs: 360

-3451.999999999933
Num buys: 144
Num sells: 175
Num trades: 319
Num wins: 85
Num losses: 233
Win streak: 3
Loss streak: 13
Avg pips risked: 0.001260062695924761
Remaining runs: 359
Best reward so far: -3451.999999999933

-5112.749999999993
Num buys: 174
Num sells: 221
Num trades: 395
Num wins: 97
Num losses: 251
Win streak: 6
Loss streak: 16
Avg pips risked: 0.0011296708860759495
Remaining runs: 358
Best reward so far: -3451.999999999933

-1588.4999999999677
Num buys: 110
Num sells: 133
Num trades: 243
Num wins: 85
Num losses: 157
Win streak: 4
Loss streak: 11
Avg pips risked: 0.0012673662551440342
Remaining runs: 357
Best reward so far: -1588.4999999999677

-4848.999999998768
Num buys: 133
Num sells: 158
Num trades: 291
Num wins: 65
Num losses: 225
Win streak: 3
Loss streak: 14
Avg pips risked: 0.0009999999999998899
Remaining runs: 356
Best reward so far: -1588.4999999999677

-3618.9999999999286
Num buys: 162
Num sells: 206
Num trades: 368
Num wins: 121
Num losses: 246


KeyboardInterrupt: 

In [ ]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best spread: ' + str(best_spread_cutoff))
print('Best TL val: ' + str(best_use_tl_val))
print('Best min pips to risk: ' + str(best_min_pips_val))
print('Best max pips to risk: ' + str(best_max_pips_val))
print('Best adx val: ' + str(best_adx_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)